In [111]:
# First prototype for an SEC filings scraper

In [ ]:
# Notes:
    # On the SEC, max requests rate is 10 per second
    # They ask to give a user agent with contact details

# URL's:
    # https://www.sec.gov/files/company_tickers_exchange.json
    # https://data.sec.gov/submissions/CIK{cik}.json
    # https://data.sec.gov/submissions/{submission_id}"
    # https://www.sec.gov/Archives/edgar/data/{cik}/{submission_id}/
    # https://www.sec.gov/Archives/edgar/data/{cik}
    # https://www.sec.gov/Archives/edgar/data/{cik}/{without_dash}/{accession_number}.txt
    # data.sec.gov/api/xbrl/companyconcept/CIK{cik}/us-gaap/{tag}.json

    # data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json

In [113]:
import time
import requests

In [114]:
last_req = time.time() # time at which last request was made

def request(url: str) -> dict:
    cur_time = time.time()
    if cur_time-last_req < 0.1: # max request rate is 10/sec
        time.sleep(0.1-(cur_time-last_req))

    req = requests.get(
        url,
        headers = {
            "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148"
        }
    )
    if req.status_code != 200:
        print(req.content.decode())
        raise RuntimeError(f"Failed request, code {req.status_code}")

    return req.json()

In [ ]:
class Filing:
    def __init__(self, accession_num: int | str, form_num: str, date: str):
        self.accession_num = str(accession_num)
        self.form_num = form_num
        self.date = date

    def get():
        pass

    def extract():
        pass

In [ ]:
class Company:
    def __init__(self, name: str, cik: str | int, ticker: str, exchange: str):
        self.name = name
        self.cik = str(cik).zfill(10)
        self.ticker = ticker
        self.exchange = exchange

    def get_filings(self): # https://data.sec.gov/submissions/CIK{cik}.json
        filings = []

        def add_to_filings(filings_json):
            num_filings = len(filings_json["accessionNumber"])
            for i in range(num_filings):
                filings.append(Filing(
                    filings_json["accessionNumber"][i],
                    filings_json["form"][i],
                    filings_json["reportDate"][i],
                ))

        filings_info = request(f"https://data.sec.gov/submissions/CIK{self.cik}.json")
        extra_filenames = [file["name"] for file in filings_info["filings"]["files"]]

        filings_json = filings_info["filings"]["recent"]
        add_to_filings(filings_json)
        for file_name in extra_filenames:
            filings_json = request(f"https://data.sec.gov/submissions/{file_name}")
            add_to_filings(filings_json)

        return filings

    def get_facts(self):
        return request(f"https://data.sec.gov/api/xbrl/companyfacts/CIK{self.cik}.json")

In [117]:
companies = []
companies_info = request("https://www.sec.gov/files/company_tickers_exchange.json")
for cik, name, ticker, exchange in companies_info["data"]:
    companies.append(Company(name, cik, ticker, exchange))

In [ ]:
############# Testing zone ###############3

In [121]:
# https://www.sec.gov/Archives/edgar/data/320193/000032019325000079/0000320193-25-000079.txt
# https://www.sec.gov/Archives/edgar/data/{cik}/{without_dash}/{accession_number}.txt
# https://www.sec.gov/Archives/edgar/data/{cik}/{submission_id}/
# https://data.sec.gov/submissions/{submission_id}"
f = Filing("000032019325000079", None, None)

In [ ]:
# TODO: f.get()